In [29]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.mixture import GaussianMixture
from scipy.stats import multivariate_normal    # 多元正态分布
from sklearn.metrics import pairwise_distances_argmin

# GaussianMixture 说明
## 参数
- n_components: int,1；混合最合的个数，可以简单理解为聚类或分类的个数
- covariance_type：协方差矩阵类型，可选
   - 'full': 每个组件都有自己的公用的协防差矩阵
   - 'tied': 所有组件公用一个协方差矩阵
   - 'diag': 每个组件有其自己的对角协方差矩阵
   - 'spherical': 每个组件有其自己的方差
- tol: float,1e-3；收敛阈值，在迭代过程中，平均增益小于该值时，EM算法停止
- reg_covar：协方差对角线上的非负正则化参数，默认为0
- max_iter：em算法的最大迭代次数，默认100
- n_init: 默认值1，执行初始化操作数量，该参数最好不要变动
- init_params：初始化权重值、均值以及精度的方法，参数可选：kmeans、random，默认kmeans
- weights_init：初始化权重列表，如果没有给定，那么使用init_params参数给定的方法来进行创建，默认为None
- means_init：初始化均值列表，如果没有给定，那么使用init_params参数给定的方法来进行创建，默认为None
- precisions_init: 初始化精度列表，如果没有给定，那么使用init_params参数给定的方法来进行创建，默认为None
- warn_stat：默认为False，当该值为true的时候，在类似问题被多次训练的时候，可以加快收敛速度

## 属性
- weights_：各组件的权重
- means：各组件的均值
- covariances_：各混合组件的协方差
- precisions_：混合组件的密度矩阵
- precisions_cholesky_：
- converged_：训练过程中是否收敛
- n_iter_：最适合EM达到收敛的步数
- lower_bound_：EM算法最佳拟合的似然概率的下限

## 方法
- aic:
- bic: 模型的评估方法

In [2]:
plt.rcParams['font.sans-serif'] = ['Heiti']
plt.rcParams['axes.unicode_minus'] = False

### 使用sklearn 中的em算法，实现高斯混合模型

In [35]:
# 创建模拟数据
np.random.seed(9)
N = 400
M = 100

## 根据给定的均值和协方差矩阵构建数据
mean1 = (0, 0, 0)
cov1 = np.diag((1, 2, 3))
## 产生400条数据
data1 = np.random.multivariate_normal(mean1, cov1, N)

## 产生一个数据分布不均衡的数据集， 100条
mean2 = (2, 2, 1)
cov2 = np.array(((3, 1, 0), (1, 3, 0), (0, 0, 3)))
data2 = np.random.multivariate_normal(mean2, cov2, M)

## 合并data1和data2这两个数据集
data = np.vstack((data1, data2))

## 产生数据对应的y值
y1 = np.array([True] * N + [False] * M)
y2 = ~y1

In [36]:
gmm = GaussianMixture(
    n_components=2, 
    covariance_type='full',
    tol=1e-6,
    max_iter=1000,
    init_params='kmeans'
)
gmm.fit(data)

# 效果输出
print('类别概率:\t', gmm.weights_)
print('均值:\n', gmm.means_, '\n')
print('方差:\n', gmm.covariances_, '\n')
print('似然函数的值:\n', gmm.lower_bound_)
mu1, mu2 = gmm.means_
sigma1, sigma2 = gmm.covariances_

类别概率:	 [0.08779945 0.91220055]
均值:
 [[3.52168168 3.17687574 0.98144041]
 [0.13527918 0.23063094 0.09986717]] 

方差:
 [[[ 0.81543892 -0.227921   -0.18242806]
  [-0.227921    3.41922573  0.35167454]
  [-0.18242806  0.35167454  2.69884223]]

 [[ 1.08575813  0.21201339  0.07229798]
  [ 0.21201339  2.48071071  0.12577066]
  [ 0.07229798  0.12577066  3.1391171 ]]] 

似然函数的值:
 -5.56934500696442


In [37]:
norm1 = multivariate_normal(mu1, sigma1)
norm2 = multivariate_normal(mu2, sigma2)
tau1 = norm1.pdf(data)
tau2 = norm2.pdf(data)

In [38]:
## 计算均值的距离，然后根据距离得到分类情况
dist = pairwise_distances_argmin([mean1, mean2], [mu1, mu2], metric='euclidean')
print ("距离:", dist)
if dist[0] == 0:
    c1 = tau1 > tau2
else:
    c1 = tau1 < tau2
c2 = ~c1

## 计算准确率
acc = np.mean(y1 == c1)
print (u'准确率：%.2f%%' % (100*acc))

距离: [1 0]
准确率：89.40%
